# Assignment: Scalable Processing
## Yelp Reviews and Authenticity

Large Scale Data Analysis | by ___ | ____@itu.dk | date

## Connecting to the Spark Cluster job using the two JobParameters.json

To connect this jupyter notebook with your Spark cluster, we need to tell jupyter how it can access the spark cluster. Below code accomplishes that. 

In [1]:
#####################################################################
# DO NOT CHANGE ANYTHING HERE.
# IF YOU HAVE PROBLEMS, CHECK THE ASSIGNMENT GUIDE CAREFULLY 
#####################################################################
    
# Only execute this cell once.
if '_EXECUTED_' in globals():
    # check if variable '_EXECUTED_' exists in the global variable namespace
    print("Already been executed once, not running again!")
else:
    print("Cell has not been executed before, running...")
    import os, json, pyspark
    from pyspark.conf import SparkConf
    from pyspark.sql import SparkSession, functions as F

    # Two files are automatically read: JobParameters.json for the Spark Cluster job using a temporary spark instance
    # and JobParameters.json for the Jupyter Lab job to extract the hostname of the cluster. 

    MASTER_HOST_NAME = None

    # Open the parameters Jupyter Lab app was launched with
    with open('/work/JobParameters.json', 'r') as file:
        JUPYTER_LAB_JOB_PARAMS = json.load(file)
        # from pprint import pprint; pprint(JUPYTER_LAB_JOB_PARAMS) 
        for resource in JUPYTER_LAB_JOB_PARAMS['request']['resources']:
            if 'hostname' in resource.keys():
                MASTER_HOST_NAME = resource['hostname']

    MASTER_HOST = f"spark://{MASTER_HOST_NAME}:7077"

    conf = SparkConf().setAll([
            ("spark.app.name", 'reading_job_params_app'), 
            ("spark.master", MASTER_HOST),
        ])
    spark = SparkSession.builder.config(conf=conf)\
                                .getOrCreate()

    CLUSTER_PARAMETERS_JSON_DF = spark.read.option("multiline","true").json('/work/JobParameters.json')

    # Extract cluster info from the specific JobParameters.json
    NODES = CLUSTER_PARAMETERS_JSON_DF.select("request.replicas").first()[0]
    CPUS_PER_NODE = CLUSTER_PARAMETERS_JSON_DF.select("machineType.cpu").first()[0] - 1
    MEM_PER_NODE = CLUSTER_PARAMETERS_JSON_DF.select("machineType.memoryInGigs").first()[0]

    CLUSTER_CORES_MAX = CPUS_PER_NODE * NODES
    CLUSTER_MEMORY_MAX = MEM_PER_NODE * NODES 
    
    if CPUS_PER_NODE > 1:
        EXECUTOR_CORES = CPUS_PER_NODE - 1  # set cores per executor on worker node
    else:
        EXECUTOR_CORES = CPUS_PER_NODE 

    EXECUTOR_MEMORY = int(
        MEM_PER_NODE / (CPUS_PER_NODE / EXECUTOR_CORES) * 0.5
    )  # set executor memory in GB on each worker node

    # Make sure there is a dir for spark logs
    if not os.path.exists('spark_logs'):
        os.mkdir('spark_logs')

    conf = SparkConf().setAll(
        [
            ("spark.app.name", 'spark_assignment'), # Change to your liking 
            ("spark.sql.caseSensitive", False), # Optional: Make queries strings sensitive to captialization
            ("spark.master", MASTER_HOST),
            ("spark.cores.max", CLUSTER_CORES_MAX),
            ("spark.executor.cores", EXECUTOR_CORES),
            ("spark.executor.memory", str(EXECUTOR_MEMORY) + "g"),
            ("spark.eventLog.enabled", True),
            ("spark.eventLog.dir", "spark_logs"),
            ("spark.history.fs.logDirectory", "spark_logs"),
            ("spark.deploy.mode", "cluster"),
        ]
    )

    ## check executor memory, taking into accout 10% of memory overhead (minimum 384 MiB)
    CHECK = (CLUSTER_CORES_MAX / EXECUTOR_CORES) * (
        EXECUTOR_MEMORY + max(EXECUTOR_MEMORY * 0.10, 0.403)
    )

    assert (
        int(CHECK) <= CLUSTER_MEMORY_MAX
    ), "Executor memory larger than cluster total memory!"

    # Stop previous session that was just for loading cluster params
    spark.stop()

    # Start new session with above config, that has better resource handling
    spark = SparkSession.builder.config(conf=conf)\
                                .getOrCreate()
    sc = spark.sparkContext
    _EXECUTED_ = True
    print("Success!")

Cell has not been executed before, running...
Success!


Click on the "SparkMonitor" tab at the top in Jupyter Lab to see the status of running code on the cluster.

## Loading the data
Here we specify where the yelp datasets are located on UCloud and read then using the spark session.

In [2]:
# Read in the business and review files
# This is the path to the shared datasets provided by adding an the dataset input folder
# when submitting the spark cluster job.
business = spark.read.json('file:////work/yelp/yelp_academic_dataset_business.json') # Use the file:/// prefix to indicate we want to read from the cluster's filesystem
business = business.persist()
# Persist 2 commonly used dataframes since they're used for later computations
# https://sparkbyexamples.com/spark/spark-difference-between-cache-and-persist/

users = spark.read.json("file:////work/yelp/yelp_academic_dataset_user.json")

reviews = spark.read.json('file:////work/yelp/yelp_academic_dataset_review.json')
reviews = reviews.persist()

In [3]:
# Get number of rows with no sampling:
reviews.count()

6990280

In [4]:
# OPTIONAL:
# Reduce resource usage and make queries run faster
# by only using a small sample of the dataframe
# and overwriting previous variable "df".
# Useful while developing, not so much to
# provide final answers. Therefore: Remember to 
# to re-read the df when done developing code using
# df = spark.read etc like above.
# reviews = reviews.sample(withReplacement=False, fraction=1/50)

# Get number of rows after sampling:
reviews.count() 

6990280

In [5]:
business.show()

+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Santa Barbara|                null|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|        Affton|{8:0-18:30, 0:0-0...|      1|   

In [6]:
reviews.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

Example: Say we're only interested in reviews of good mexican restaurants in Arizona. You can delete this when you do your own thing. 

In [7]:
# Filter to only Arizona businesses with "Mexican" as part of their categories
az_mex = business.filter(business.state == "AZ")\
                .filter(business.categories.rlike("Mexican"))\
                .select("business_id", "name")

# Join with the reviews
az_mex_rs = reviews.join(az_mex, on="business_id", how="inner")

# Filter to only 5 star reviews
good_az_mex_rs = az_mex_rs.filter(az_mex_rs.stars == 5)\
                        .select("name","text")

# Print the top 20 rows of the DataFrame
good_az_mex_rs.show()

# Convert to pandas (local object) and save to local file system
good_az_mex_rs.toPandas().to_csv("good_az_reviews.csv", header=True, index=False, encoding='utf-8')


+--------------------+--------------------+
|                name|                text|
+--------------------+--------------------+
|Casa Molina Del N...|We've been coming...|
|St Mary's Mexican...|Some of the fines...|
|Street- Taco and ...|Top notch street ...|
|Indian Frybread-M...|This place is a h...|
|St Mary's Mexican...|One of my favorit...|
|Street- Taco and ...|This is my favori...|
|Street- Taco and ...|Great food!  Grea...|
|            BK Tacos|Quality ingredien...|
|Street- Taco and ...|Best tacos in Tuc...|
|        El Merendero|This place was fr...|
|        El Merendero|To the reviewer w...|
|Taqueria Pico De ...|If you are lookin...|
|               Penca|Good unique Mexic...|
|   La Mesa Tortillas|Incredible red ch...|
|               Penca|I love this place...|
|            BK Tacos|This place is the...|
|Indian Frybread-M...|Wow! My sister ha...|
|St Mary's Mexican...|Carne Seca burro ...|
|             Club 21|The food was exce...|
|      El Charro Cafe|This is th

In [8]:
reviews.rdd.getNumPartitions()

40

In [9]:
reviews.repartition(25)

DataFrame[business_id: string, cool: bigint, date: string, funny: bigint, review_id: string, stars: double, text: string, useful: bigint, user_id: string]

In [10]:
# 1. Find the total number of reviews for all businesses. The output should be in the form of a Spark Table/DataFrame with one value representing the count.
# I'm gonna assuma that's "number of reviews for each business"
# https://stackoverflow.com/a/41890813/9877700
reviewCount = reviews.groupBy("business_id").agg(
    F.count(F.lit(1)).alias("num_reviews")
)
reviewCount.sort("num_reviews", ascending=False).show()

+--------------------+-----------+
|         business_id|num_reviews|
+--------------------+-----------+
|_ab50qdWOk0DdB6XO...|       7673|
|ac1AeYqs8Z4_e2X5M...|       7516|
|GXFMD0Z4jEVZBCsbP...|       6160|
|ytynqOUb3hjKeJfRj...|       5778|
|oBNrLz4EDhiscSlbO...|       5264|
|iSRTaT9WngzB8JJ2Y...|       5254|
|VQcCL9PiNL_wkGf-u...|       5146|
|_C7QiQQc47AOEv4PE...|       4969|
|GBTPC53ZrG1ZBY3DT...|       4661|
|6a4gLLFSgr-Q6CZXD...|       4480|
|PP3BBaVxZLcJU54uP...|       4293|
|1b5mnK8bMnnju_cvU...|       4247|
|I_3LMZ_1m2mzR0oLI...|       4093|
|VaO-VW3e1kARkU9bP...|       4034|
|qb28j-FNX1_6xm7u3...|       3971|
|gTC8IQ_i8zXytWSly...|       3917|
|yPSejq3_erxo9zdVY...|       3889|
|wz8ZPfySQczcPgSyd...|       3634|
|VVH6k9-ycttH3TV_l...|       3633|
|IkY2ticzHEn4QFn8h...|       3428|
+--------------------+-----------+
only showing top 20 rows



In [11]:
# 2. Find all businesses that have received 5 stars and that have been reviewed by 500 or more users. The output should be in the form of DataFrame of (name, stars, review count).
bsnsrvwcnt = reviewCount.join(business, "business_id", "inner")
bsnsrvwcnt500 = bsnsrvwcnt.filter(bsnsrvwcnt.num_reviews >= 500)
try:
    bsnsrvwcnt500.select("name", "stars", "num_reviews").show()
except Exception as e:
    print(e) # why?
    bsnsrvwcnt500.show()

An error occurred while calling o208.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 18.0 failed 4 times, most recent failure: Lost task 1.3 in stage 18.0 (TID 285) (node1 executor 0): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1182)
	at or

In [12]:
# 3. Find the influencers who have written more than 1000 reviews. The output should be in the form of a Spark Table/DataFrame of user id. Find the businesses that have been reviewed by more than 5 influencer users. You can use a view created from your answer to Q3.
# "view created from your answer to Q3" brother this IS Q3.
userReviewCount = reviews.groupBy("user_id").agg(
    F.count(F.lit(1)).alias("num_reviews")
)
userReviewCount.sort("num_reviews", ascending=False).show()

+--------------------+-----------+
|             user_id|num_reviews|
+--------------------+-----------+
|_BcWyKQL16ndpBdgg...|       3048|
|Xw7ZjaGfr0WNVt6s_...|       1840|
|0Igx-a1wAstiBDerG...|       1747|
|-G7Zkl1wIWBBmD0KR...|       1682|
|ET8n-r7glWYqZhuR6...|       1653|
|bYENop4BuQepBjM1-...|       1578|
|1HM81n6n4iPIFU5d2...|       1554|
|fr1Hz2acAb3OaL3l6...|       1447|
|wXdbkFZsfDR7utJvb...|       1396|
|Um5bfs5DH6eizgjH3...|       1391|
|qjfMBIZpQT9DDtw_B...|       1324|
|VL12EhEdT4OWqGq0n...|       1308|
|bJ5FtCtZX3ZZacz2_...|       1298|
|pou3BbKsIozfH50rx...|       1247|
|ouODopBKF3AqfCkuQ...|       1129|
|B-s-8VUnuBjGTP3d0...|       1087|
|-kLVfaJytOJY2-QdQ...|       1076|
|vHc-UrI9yfL_pnnc6...|       1071|
|CfX4sTIFFNaRchNsw...|       1047|
|AHRrG3T1gJpHvtpZ-...|       1041|
+--------------------+-----------+
only showing top 20 rows



In [13]:
influencers = userReviewCount.filter(userReviewCount.num_reviews > 1000).select("user_id")
influencers.show()

+--------------------+
|             user_id|
+--------------------+
|1HM81n6n4iPIFU5d2...|
|-kLVfaJytOJY2-QdQ...|
|B-s-8VUnuBjGTP3d0...|
|Xw7ZjaGfr0WNVt6s_...|
|bYENop4BuQepBjM1-...|
|0Igx-a1wAstiBDerG...|
|CfX4sTIFFNaRchNsw...|
|pou3BbKsIozfH50rx...|
|ouODopBKF3AqfCkuQ...|
|qjfMBIZpQT9DDtw_B...|
|-G7Zkl1wIWBBmD0KR...|
|AHRrG3T1gJpHvtpZ-...|
|wXdbkFZsfDR7utJvb...|
|vHc-UrI9yfL_pnnc6...|
|VL12EhEdT4OWqGq0n...|
|_BcWyKQL16ndpBdgg...|
|bJ5FtCtZX3ZZacz2_...|
|ET8n-r7glWYqZhuR6...|
|I2XpWCHAom1JRyHXZ...|
|fr1Hz2acAb3OaL3l6...|
+--------------------+
only showing top 20 rows



In [14]:
influencerReviews = reviews.join(influencers, "user_id", "inner")
influencerReviewCount = influencerReviews.groupBy("business_id").agg(
    F.count(F.lit(1)).alias("num_influencer_reviews")
)
influencerReviewCount.show()

+--------------------+----------------------+
|         business_id|num_influencer_reviews|
+--------------------+----------------------+
|h-LoRnHxlAx4tS2UJ...|                     2|
|qWxW41eUYVOO79YP7...|                     1|
|y6aBgF8SQ3PpR6AaW...|                     3|
|KdljisvY2nmyHSIUt...|                     1|
|PQYuPr2Dfm2S49EUL...|                     5|
|xoVTtQd6xRrFbQcdf...|                     3|
|CG_NAd2OggmVEQjFO...|                     1|
|6nRTFCfS3CsIWyqOj...|                     2|
|AlnAoqsqChTn1Eg3d...|                     1|
|1u2IlvwOMndfuRk9E...|                     2|
|lXCFcmhoRsyW-mnzz...|                     1|
|P6fQZruWG886KpegS...|                     2|
|2j43r7yLc-KwJ6JNs...|                     1|
|xZvfws735zT0LsBZM...|                     1|
|GApqfwJMxg4t2tk3K...|                     3|
|uAg0M9xi_ijuMw5T2...|                     4|
|2N97axWHko3rzyzOG...|                     2|
|PQ0QbT5I6V8JvOm0N...|                     4|
|WeDn7iD1ckcEg7Ytt...|            

In [15]:
popularBusinesses = influencerReviewCount.filter(influencerReviewCount.num_influencer_reviews > 5)
popularBusinesses.show()

+--------------------+----------------------+
|         business_id|num_influencer_reviews|
+--------------------+----------------------+
|GjQPosKRsjgy6YOCl...|                     6|
|QHWYlmVbLC3K6eglW...|                     8|
|PP3BBaVxZLcJU54uP...|                     6|
|N2dOYDp9aKOBxyO2e...|                     7|
|G_bi7l0nU74I4Vr1V...|                     9|
|O1Cw2yzf4bCuKjbVT...|                     6|
|I_3LMZ_1m2mzR0oLI...|                     7|
|a8CrdVtlfa3JDoC_A...|                     6|
|Eb1XmmLWyt_way5NN...|                     6|
|Uw46n__imJ52D7Zh1...|                     6|
|g0cUlBQ2pGRxWtL6q...|                     7|
|c_4c5rJECZSfNgFj7...|                     8|
|Hi2ADMI1_BEahkXRP...|                     8|
|n_7EQxn0ciucIOJfp...|                     7|
|ny1N_Py01kVoGOvL9...|                     6|
|7jRXF4N5NzxYuNN-H...|                     7|
|KhBUg5QhBYuK8RZAe...|                     6|
|AFYI0sfZ6WdVELjjE...|                     7|
|_0QZGrg91aaiMhh7t...|            

In [16]:
# 4. Find an ordered list of usrs based on the average star counts they have given in all their reviews.
userMeanStars = reviews.groupBy("user_id").agg(
    F.mean("stars").alias("mean_stars")
).sort("mean_stars", ascending=False)
userMeanStars.show()

+--------------------+----------+
|             user_id|mean_stars|
+--------------------+----------+
|gu9gCRLNtQYuaQ2j_...|       5.0|
|4JpCEIxTvZIIT9zDM...|       5.0|
|6Bo23tnTkx6zN7_YG...|       5.0|
|kbx5teHlHE1D0WMtp...|       5.0|
|7hVPUkpLOgRuRNlTf...|       5.0|
|y2pb6ko7InuUkhMBN...|       5.0|
|q8_2bjtFhtKZO1yU9...|       5.0|
|Hl9WmJ1t2xMF-u1-r...|       5.0|
|HrigG63kjiYOrRpSq...|       5.0|
|egWm6MlMfJYIbIv8c...|       5.0|
|KqbJWGy8o1EQeTKJh...|       5.0|
|Qqdjv0GaMuqMbBeYr...|       5.0|
|wo2V_OfgU0NBDPUzE...|       5.0|
|h3E1Z4x1Jkxamxi21...|       5.0|
|A3EiqW7_k00gvaiQi...|       5.0|
|sm4ZPuv-HpGBn3CPW...|       5.0|
|JHWGK0kIWM73UcpO2...|       5.0|
|3rSKIFAWcJ-0jkRJc...|       5.0|
|22ml-CTcoabnc-uu4...|       5.0|
|kALjfwXfePTGEpai5...|       5.0|
+--------------------+----------+
only showing top 20 rows



In [17]:
# 3.2: Authenticity study
# 3.2.1: Data Exploration
from functools import reduce
from pyspark.sql.functions import lower, udf
authent_words = ["authentic", "veritable", "legitimate"]
authentic_reviews = reviews.where(
    reduce(lambda a, b: a|b, (lower(reviews.text).like('%' + word + '%') for word in authent_words))
)
num_authentic = authentic_reviews.rdd.count()
num_total = reviews.rdd.count()
print(num_authentic)

129945


In [18]:
# - What is the percentage of reviews containing a variant of the word "authentic"?
print(f"{(num_authentic / num_total) * 100:.2f}% of reviews mention authenticity")

1.86% of reviews mention authenticity


In [19]:
# - How many reviews contain the string "legitimate" grouped by type of cuisine?
from pyspark.sql.types import StringType
authentic_reviews_business = authentic_reviews.join(business, on="business_id")
cuisines = [
    "Indian", "Taiwanese", "Chinese", "American (Traditional)", "American (New)", "French", "Mexican", "Vietnamese", "Lebanese", "Greek", "Italian", "Trinidadian",
    "Filipino", "Cuban", "Korean", "Thai", "Japanese", "Hawaiian", "Latin American", "Peruvian", "Spanish", "Ukrainian", "Irish", "Brazilian", "Senegalese",
    "Argentine", "German", "Sri Lankan", "Soul Food", "Ethiopian", "Caribbean", "Mediterranean", "African", "Middle Eastern"
]
# This function was in part created by ChatGPT
def find_cuisine(categories):
    for cuisine in cuisines:
        if cuisine in categories:
            return cuisine
    return None
find_cuisine_udf = udf(find_cuisine, StringType())

reviews_business = reviews.join(business, on="business_id")
reviews_business_cuisines = reviews_business.withColumn("cuisine", find_cuisine_udf(reviews_business.categories))
reviews_business_cuisines_food = reviews_business_cuisines.filter(lower(reviews_business_cuisines.categories).contains('food'))
authentic_reviews_business_cuisines_food = reviews_business_cuisines_food.where(
    reduce(lambda a, b: a|b, (lower(reviews_business_cuisines_food.text).like('%' + word + '%') for word in authent_words))
)
authentic_reviews_count_by_cuisine = authentic_reviews_business_cuisines_food.groupBy("cuisine").agg(
    F.count(F.lit(1)).alias("num_reviews_authentic")
).sort("cuisine")
authentic_reviews_count_by_cuisine.sort("num_reviews_authentic", ascending=False).show(authentic_reviews_count_by_cuisine.rdd.count())

+--------------------+---------------------+
|             cuisine|num_reviews_authentic|
+--------------------+---------------------+
|                null|                 9915|
|             Mexican|                 9741|
|             Chinese|                 4234|
|             Italian|                 3406|
|American (Traditi...|                 3041|
|      American (New)|                 2137|
|              Indian|                 1842|
|          Vietnamese|                 1629|
|               Greek|                 1113|
|            Japanese|                 1059|
|                Thai|                 1026|
|               Cuban|                  871|
|      Latin American|                  837|
|           Soul Food|                  677|
|              Korean|                  629|
|       Mediterranean|                  627|
|              German|                  617|
|              French|                  583|
|           Taiwanese|                  448|
|         

In [20]:
# - Is there a difference in the amount of authenticity language used in the different areas?
reviews_count_by_cuisine = reviews_business_cuisines_food.groupBy("cuisine").agg(
    F.count(F.lit(1)).alias("num_reviews_all")
).sort("cuisine")
# reviews_count_by_cuisine.sort("num_reviews_all", ascending=False).show(reviews_count_by_cuisine.rdd.count())
reviews_counts_percentages = authentic_reviews_count_by_cuisine\
    .join(reviews_count_by_cuisine, "cuisine")\
    .withColumn("reviews_percent_authentic", (F.col("num_reviews_authentic") / F.col("num_reviews_all")))
reviews_counts_percentages.show(reviews_counts_percentages.rdd.count())

+--------------------+---------------------+---------------+-------------------------+
|             cuisine|num_reviews_authentic|num_reviews_all|reviews_percent_authentic|
+--------------------+---------------------+---------------+-------------------------+
|             Mexican|                 9741|         129678|      0.07511682783509925|
|           Ethiopian|                  121|           1463|      0.08270676691729323|
|                Thai|                 1026|          15123|      0.06784368180916485|
|              Indian|                 1842|          25854|       0.0712462288233929|
|             Chinese|                 4234|          69490|     0.060929630162613324|
|           Soul Food|                  677|          32012|     0.021148319380232414|
|             African|                   31|            575|      0.05391304347826087|
|           Taiwanese|                  448|           6131|      0.07307127711629424|
|           Ukrainian|                   74

In [21]:
temp = reviews_counts_percentages\
    .select("cuisine", "num_reviews_all", "reviews_percent_authentic")\
    .withColumn("reviews_percent_authentic", (F.col("reviews_percent_authentic") * 100))\
    .sort("reviews_percent_authentic", ascending=False)
temp.show(temp.rdd.count())

+--------------------+---------------+-------------------------+
|             cuisine|num_reviews_all|reviews_percent_authentic|
+--------------------+---------------+-------------------------+
|           Ukrainian|            418|       17.703349282296653|
|         Trinidadian|            499|       12.224448897795591|
|              German|           5477|       11.265291217819975|
|            Peruvian|            853|       10.550996483001173|
|           Ethiopian|           1463|        8.270676691729323|
|            Filipino|           2241|        8.121374386434628|
|           Caribbean|           4842|        7.806691449814126|
|             Mexican|         129678|        7.511682783509925|
|           Taiwanese|           6131|        7.307127711629424|
|            Lebanese|           2176|       7.2610294117647065|
|              Indian|          25854|         7.12462288233929|
|               Cuban|          12346|        7.054916572169123|
|          Sri Lankan|   

In [29]:
# 3.2.2: Hypothesis Testing
# - Can you identify a difference in the relationship between authenticity
#   language3 and typically negative words 4, in restaurants serving south
#   american or asian cuisine comparedto restaurants serving european
#   cuisine? And to what degree?
from pyspark.sql.functions import col, when
neg_words = [ "dirty", "kitsch", "cheap", "rude", "simple", "bland", "dodgy", "poisoning" ]
pos_words = [ "clean", "refined", "elegant", "stylish" ]
reviews_business_cuisines = reviews_business_cuisines.withColumn("negative_count",
    sum(when(condition, 1).otherwise(0) for condition in [lower(col("text")).like(f"%{word}%") for word in neg_words])
).withColumn("positive_count",
    sum(when(condition, 1).otherwise(0) for condition in [lower(col("text")).like(f"%{word}%") for word in pos_words])
)
reviews_business_cuisines.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+-------+----------+-----------+-----------------+-----------+------------+-----+-----+--------------+--------------+--------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|         address|          attributes|          categories|           city|               hours|is_open|  latitude|  longitude|             name|postal_code|review_count|stars|state|       cuisine|negative_count|positive_count|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+-------+----------+-----------+-----------------+-----------+

In [43]:
reviews_business_cuisines = reviews_business_cuisines\
    .withColumn("sentiment",
         when(col("negative_count") > col("positive_count"), "negative")
        .when(col("negative_count") < col("positive_count"), "positive")
        .otherwise("neutral")
    )\
    .withColumn("authentic",
        reduce(lambda a, b: a|b, (lower(reviews_business_cuisines.text).like('%' + word + '%') for word in authent_words))
    )\
    .withColumn("isFood",
        lower(reviews_business_cuisines.categories).like("%food%")
    )
reviews_business_cuisines.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+-------+----------+-----------+-----------------+-----------+------------+-----+-----+--------------+--------------+--------------+---------+---------+------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|         address|          attributes|          categories|           city|               hours|is_open|  latitude|  longitude|             name|postal_code|review_count|stars|state|       cuisine|negative_count|positive_count|sentiment|authentic|isFood|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+-------

In [60]:
from pyspark.sql.functions import count
sentiment_by_cuisine = reviews_business_cuisines\
    .groupBy("cuisine")\
    .agg(F.count(F.when(F.col("authentic"), True)).alias("n_authentic"),
         F.count(F.lit(1)).alias("n_total"),
         F.count(F.when(F.col("sentiment") == "negative", True)).alias("n_total_negative"),
         F.count(F.when((F.col("sentiment") == "negative") & F.col("authentic"), True)).alias("n_authentic_negative"),
         F.count(F.when(F.col("sentiment") == "positive", True)).alias("n_total_positive"),
         F.count(F.when((F.col("sentiment") == "positive") & F.col("authentic"), True)).alias("n_authentic_positive"),
         F.count(F.when(F.col("sentiment") == "neutral", True)).alias("n_total_neutral"),
         F.count(F.when((F.col("sentiment") == "neutral") & F.col("authentic"), True)).alias("n_authentic_neutral"))



Py4JJavaError: An error occurred while calling o11908.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 171.0 failed 4 times, most recent failure: Lost task 0.3 in stage 171.0 (TID 1744) (node1 executor 0): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGSch